# CSCM72 Coursework 1: Python Optimisers

In [25]:
import numpy as np
import math

In [26]:
example_array = np.array([1,2,3,4])

In [27]:
# Constants to improve readability
HEIGHT = 0     #x1
LENGTH = 1     #x2
THICKNESS = 2  #x3
BREADTH = 3    #x4

### Objective Function *f(x)*:

In [28]:
def objective_calc(design):
    return ( 1.0471 * (design[HEIGHT] ** 2) * design[LENGTH]) + (0.04811 * design[THICKNESS] * design[BREADTH] * (14.0 + design[LENGTH]) )



In [29]:
objective_calc(example_array)

11.331320000000002

### Shear Stress *g<sub>1</sub>(x)*:

In [43]:
def shear_stress(design):
    # Calculates the first derivative
    tau1 = 6000 / (math.sqrt(2) * design[HEIGHT] * design[LENGTH])

    # Calculates the second derivative
    tau2 = (6000 * (14 + 0.5 * design[LENGTH]) * math.sqrt(0.25 * ((design[LENGTH] ** 2) + (design[HEIGHT] + design[THICKNESS]) ** 2 ) ) )  \
                        / ( 2 * (0.707 * design[HEIGHT] * design[LENGTH] * ( ( (design[LENGTH] ** 2) / 12) + 0.25 * (design[HEIGHT] + design[THICKNESS]) ) ) )
    
    # Calculates the shear stress of the design
    tau = math.sqrt( (tau1 ** 2 + tau2 ** 2) + ( (design[LENGTH] * tau1 * tau2) \
           / (math.sqrt(0.25*(design[LENGTH]**2 + (design[HEIGHT] + design[THICKNESS])**2))) ))

    # Returns false if shear stress exceeds 13,600psi target
    return (13600 - tau) >= 0

In [44]:
shear_stress(example_array)

False

### Normal Stress *g<sub>2</sub>(x)*:

In [45]:
def normal_stress(design):
    # Calculates normal stress of the design
    sigma = 504000 / (design[THICKNESS]**2 * design[BREADTH])

    # Returns false if normal stress exceeds the target yield strength (30,000psi)
    return (30000 - sigma) >= 0

In [46]:
normal_stress(example_array)

True

### Practicality *g<sub>3</sub>(x)*:

In [47]:
def practicality(design):
    return (design[BREADTH] - design[HEIGHT])

In [48]:
practicality(example_array)

True

### Buckling Load *g<sub>4</sub>(x)*:

In [ ]:
def buckling_load(design):
    rho = 64746.022 * (1 - 0.0282346 * design[THICKNESS]) * design[THICKNESS] * design[BREADTH]**3

    # Returns false if the buckling load is less than the target applied load (6000lbs)
    return (rho - 6000) >= 0

In [ ]:
buckling_load(example_array)